In [52]:
import pandas as pd
import glob
import os
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

files = glob.glob("HS/DOG/csv/*.csv")


df1 = pd.read_csv(files[0], sep="\t")

df_op = pd.read_csv("Operon_overlap.txt", sep="\t")
list(df1)
files

df_op["DOG_operon_same_strand"] = np.where(df_op["RUNON_strand"]==df_op["OPERON_strand"], True, False)

df_op = df_op[df_op["DOG_operon_same_strand"]==True]
df_op.head()

,RUNON,chr,RUNON_strand,RUNON_start,RUNON_end,OPERON,OPERON_strand,OPERON_start,OPERON_end,DOG_operon_same_strand
0,WBGene00021925,IV,-,1003400,1003600,CEOP4044,-,992240,1003639,True
1,WBGene00021925,IV,-,1003400,1003600,CEOP4642,-,1003600,1009734,True
3,WBGene00007902,IV,-,8773552,8773952,CEOP4308,-,8767688,8780872,True
10,WBGene00020097,III,-,5008830,5008933,CEOP3252,-,5003197,5017860,True
12,WBGene00022599,III,-,5273471,5273800,CEOP3276,-,5268682,5274787,True


In [53]:
def parse_df(f1):
    

    name = os.path.basename(f1)
    df = pd.read_csv(f1, sep="\t")
    elist = ["J2","INP"]

    enrich = ""
    for e in elist:
        if e in name:
            enrich = e

    tlist = ["N2","HS","OK"]
    treatment = ""
    for t in tlist:
        if t in name:
            treatment = t        
            

        

            
            
    df = df.sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
    print("*****")
    print("f1 is", f1)
    print(len(df))
    df= df[~df["gene_id_name"].isin(df_op["RUNON"])]    
    print(len(df))
    
    
    df = df[["sample_name","DOG_length", "gene_id_name", 'DOG_start_local','DOG_end_local',"chr",'strand',"TYPE"]]
    df["Enrichment"] = enrich 
    df["Treatment"] = treatment
    df = df[df["TYPE"]=="DOG"]
    df = df[df["DOG_length"]>100]
    df["sample_name"] = df["sample_name"].str.replace("__split","")
    df["sample_name"] = df["sample_name"].str.replace("_split","")
    df["sample_name"] = df["sample_name"].str.replace("_spl","")
    df["sample_name"] = df["sample_name"].str.replace("_sp","")
    df["sample_name"] = df["sample_name"].str.replace("_s","")
    df["sample_name"] = df["sample_name"].str.replace("-","_")
    
    df.to_csv(f1[:-4]+"_no_operon.tsv",sep="\t", index=None)
    
    return df


In [54]:

def get_venns(df,df2,col,title):
    dfAb= df[~df[col].isin(df2[col])]
    dfaB= df2[~df2[col].isin(df[col])]
    dfAB= df[df[col].isin(df2[col])]
    
    print(title)
    print("df ", len(df))
    print("df2 ", len(df2))
    
    
    print("1 ", len(dfAb))
    print("2 ", len(dfaB))
    print("both ", len(dfAB))
    
    print("ALL: ", len(dfAb)+len(dfaB)+len(dfAB))
    print("******************")
    
    
    

In [55]:
df_final = pd.DataFrame()

for f in files:
    df = parse_df(f)
    df_final = df_final.append(df)
df = df_final

*****
f1 is HS/DOG/csv/HS_INP_split_min_gtf_DOG.csv
17
16
*****
f1 is HS/DOG/csv/HS_INP_split_plu_gtf_DOG.csv
17
17
*****
f1 is HS/DOG/csv/HS_J2A_split_min_gtf_DOG.csv
210
179
*****
f1 is HS/DOG/csv/HS_J2A_split_plu_gtf_DOG.csv
252
206
*****
f1 is HS/DOG/csv/HS_J2B_split_min_gtf_DOG.csv
63
53
*****
f1 is HS/DOG/csv/HS_J2B_split_plu_gtf_DOG.csv
87
67
*****
f1 is HS/DOG/csv/N2_J2_1__split_min_gtf_DOG.csv
73
58
*****
f1 is HS/DOG/csv/N2_J2_1__split_plu_gtf_DOG.csv
70
50
*****
f1 is HS/DOG/csv/N2_J2_2__split_min_gtf_DOG.csv
62
51
*****
f1 is HS/DOG/csv/N2_J2_2__split_plu_gtf_DOG.csv
68
50
*****
f1 is HS/DOG/csv/N2_J2_3__split_min_gtf_DOG.csv
71
62
*****
f1 is HS/DOG/csv/N2_J2_3__split_plu_gtf_DOG.csv
85
60
*****
f1 is HS/DOG/csv/N2WT_INP_split_min_gtf_DOG.csv
53
41
*****
f1 is HS/DOG/csv/N2WT_INP_split_plu_gtf_DOG.csv
60
45


In [56]:
sample_list = df["sample_name"].unique().tolist()
treat_list = df["Treatment"].unique().tolist()
enrich_list = df["Enrichment"].unique().tolist()
sample_list

['N2HS_INP',
 'N2HS_J2A',
 'N2HS_J2B',
 'N2_J2_1',
 'N2_J2_2',
 'N2_J2_3',
 'N2WT_INP']

In [58]:
g = df.groupby(by="sample_name", as_index=False)
s = g.size()
df_size = s.to_frame()
df_size = df_size.rename( columns={0: "Sample size"})
df_size["Sample"] = df_size.index
df_size

,Sample size,Sample
sample_name,,
N2HS_INP,31,N2HS_INP
N2HS_J2A,385,N2HS_J2A
N2HS_J2B,119,N2HS_J2B
N2WT_INP,86,N2WT_INP
N2_J2_1,104,N2_J2_1
N2_J2_2,100,N2_J2_2
N2_J2_3,122,N2_J2_3


In [59]:
df_TO    = df[ df["Enrichment"]=="INP"]
df_J2    = df[ df["Enrichment"]=="J2" ]
df_HS    = df[ df["Treatment"]=="HS"]
df_N2    = df[ df["Treatment"]=="N2"]
df_HS_J2 = df_J2[ df_J2["Treatment"]=="HS" ]
df_HS_TO = df_TO[df_TO["Treatment"]=="HS"]
df_N2_J2 = df_J2[ df_J2["Treatment"]=="N2" ]   
df_N2_TO = df_TO[df_TO["Treatment"]=="N2"]     
df_ALL = df



df_TO    = df_TO.sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_J2    = df_J2.sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_HS    = df_HS.sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_N2    = df_N2.sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_HS_J2 = df_HS_J2.sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_HS_TO = df_HS_TO.sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_N2_J2 = df_N2_J2.sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_N2_TO = df_N2_TO.sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_ALL = df_ALL.sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")

print("After dropping duplicates")

print("df_TO\t", len(df_TO))         
print("df_J2\t", len(df_J2))         
print("df_HS\t", len(df_HS))         
print("df_N2\t", len(df_N2))         
print("df_HS_J2\t", len(df_HS_J2))   
print("df_HS_TO\t", len(df_HS_TO))   
print("df_N2_J2\t", len(df_N2_J2))   
print("df_N2_TO\t", len(df_N2_TO))   
print("df ALL", len(df_ALL))         

After dropping duplicates
df_TO	 111
df_J2	 474
df_HS	 398
df_N2	 194
df_HS_J2	 396
df_HS_TO	 31
df_N2_J2	 169
df_N2_TO	 86
df ALL 490


In [60]:
get_venns(df_HS_J2,df_N2_J2,col="gene_id_name",title="J2: HS vs N2")
get_venns(df_HS_TO,df_N2_TO,col="gene_id_name",title="TO: HS vs N2")
get_venns(df_TO,df_J2,col="gene_id_name",title="TOTAL vs J2: HS vs N2")

get_venns(df_HS_TO,df_HS_J2,col="gene_id_name",title="HS: TO vs J2")
get_venns(df_N2_TO,df_N2_J2,col="gene_id_name",title="N2: TO vs J2")

J2: HS vs N2
df  396
df2  169
1  305
2  78
both  91
ALL:  474
******************
TO: HS vs N2
df  31
df2  86
1  25
2  80
both  6
ALL:  111
******************
TOTAL vs J2: HS vs N2
df  111
df2  474
1  16
2  379
both  95
ALL:  490
******************
HS: TO vs J2
df  31
df2  396
1  2
2  367
both  29
ALL:  398
******************
N2: TO vs J2
df  86
df2  169
1  25
2  108
both  61
ALL:  194
******************


In [61]:
# Do the same for OK803

files = glob.glob("OK/DOG/csv/*.csv")

df_final = pd.DataFrame()

for f in files:
    df = parse_df(f)
    df_final = df_final.append(df)
df = df_final

sample_list = df["sample_name"].unique().tolist()
treat_list = df["Treatment"].unique().tolist()
enrich_list = df["Enrichment"].unique().tolist()
enrich_list

*****
f1 is OK/DOG/csv/N2-J2-1__split_min_gtf_DOG.csv
73
58
*****
f1 is OK/DOG/csv/N2-J2-1__split_plu_gtf_DOG.csv
70
50
*****
f1 is OK/DOG/csv/N2-J2-2__split_min_gtf_DOG.csv
62
51
*****
f1 is OK/DOG/csv/N2-J2-2__split_plu_gtf_DOG.csv
68
50
*****
f1 is OK/DOG/csv/N2-J2-3__split_min_gtf_DOG.csv
71
62
*****
f1 is OK/DOG/csv/N2-J2-3__split_plu_gtf_DOG.csv
85
60
*****
f1 is OK/DOG/csv/N2WT_INP_split_min_gtf_DOG.csv
53
41
*****
f1 is OK/DOG/csv/N2WT_INP_split_plu_gtf_DOG.csv
60
45
*****
f1 is OK/DOG/csv/OK8-J2-1_split_min_gtf_DOG.csv
62
51
*****
f1 is OK/DOG/csv/OK8-J2-1_split_plu_gtf_DOG.csv
72
49
*****
f1 is OK/DOG/csv/OK8-J2-2_split_min_gtf_DOG.csv
61
48
*****
f1 is OK/DOG/csv/OK8-J2-2_split_plu_gtf_DOG.csv
65
42
*****
f1 is OK/DOG/csv/OK8-J2-3_split_min_gtf_DOG.csv
36
27
*****
f1 is OK/DOG/csv/OK8-J2-3_split_plu_gtf_DOG.csv
54
37
*****
f1 is OK/DOG/csv/OK80-INP_split_min_gtf_DOG.csv
119
95
*****
f1 is OK/DOG/csv/OK80-INP_split_plu_gtf_DOG.csv
130
98


['J2', 'INP']

In [62]:
g = df.groupby(by="sample_name", as_index=False)
s = g.size()
df_size = s.to_frame()
df_size = df_size.rename( columns={0: "Sample size"})
df_size["Sample"] = df_size.index
df_size

,Sample size,Sample
sample_name,,
N2WT_INP,86,N2WT_INP
N2_J2_1,104,N2_J2_1
N2_J2_2,100,N2_J2_2
N2_J2_3,122,N2_J2_3
OK80_INP,193,OK80_INP
OK8_J2_1,100,OK8_J2_1
OK8_J2_2,89,OK8_J2_2
OK8_J2_3,64,OK8_J2_3


In [63]:
df_TO = df[ df["Enrichment"]=="INP"]
df_J2 = df[ df["Enrichment"]=="J2" ]
df_OK = df[ df["Treatment"]=="OK"]
df_N2 = df[ df["Treatment"]=="N2"]
df_OK_J2 = df[ (df["Treatment"]=="OK") & (df["Enrichment"]=="J2") ]
df_OK_TO = df[ (df["Treatment"]=="OK") & (df["Enrichment"]=="INP") ]
df_N2_J2 = df[ (df["Treatment"]=="N2") & (df["Enrichment"]=="J2") ]
df_N2_TO = df[ (df["Treatment"]=="N2") & (df["Enrichment"]=="INP") ]
df_all = df


# print("df_TO ", len(df_TO))    
# print("df_J2 ", len(df_J2))    
# print("df_OK ", len(df_OK))    
# print("df_N2 ", len(df_N2))    
# print("df_OK_J2 ", len(df_OK_J2)) 
# print("df_OK_TO ", len(df_OK_TO)) 
# print("df_N2_J2 ", len(df_N2_J2)) 
# print("df_N2_TO ", len(df_N2_TO)) 
# print("dfall ", len(df_ALL)) 


df_TO = df[ df["Enrichment"]=="INP" ].sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_J2 = df[ df["Enrichment"]=="J2" ].sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")

df_OK = df[ df["Treatment"]=="OK"].sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_N2 = df[ df["Treatment"]=="N2"].sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_ALL = df.sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")

df_OK_J2 = df[ (df["Treatment"]=="OK") & (df["Enrichment"]=="J2") ].sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_OK_TO = df[ (df["Treatment"]=="OK") & (df["Enrichment"]=="INP") ].sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_N2_J2 = df[ (df["Treatment"]=="N2") & (df["Enrichment"]=="J2") ].sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")
df_N2_TO = df[ (df["Treatment"]=="N2") & (df["Enrichment"]=="INP") ].sort_values('DOG_length', ascending=False).drop_duplicates("gene_id_name")

print("After dropping duplicates")


print("df_TO ", len(df_TO))    
print("df_J2 ", len(df_J2))    
print("df_OK ", len(df_OK))    
print("df_N2 ", len(df_N2))    
print("df_OK_J2 ", len(df_OK_J2)) 
print("df_OK_TO ", len(df_OK_TO)) 
print("df_N2_J2 ", len(df_N2_J2)) 
print("df_N2_TO ", len(df_N2_TO)) 
print("dfall ", len(df_ALL)) 


After dropping duplicates
df_TO  211
df_J2  196
df_OK  237
df_N2  194
df_OK_J2  128
df_OK_TO  193
df_N2_J2  169
df_N2_TO  86
dfall  300


In [64]:

get_venns(df_OK_J2,df_N2_J2,col="gene_id_name",title="J2: OK vs N2")
get_venns(df_OK_TO,df_N2_TO,col="gene_id_name",title="TO: OK vs N2")
get_venns(df_TO,df_J2,col="gene_id_name",title="TO vs J2: OK vs N2")

get_venns(df_OK_TO,df_OK_J2,col="gene_id_name",title="OK vs OK: TO vs J2")


J2: OK vs N2
df  128
df2  169
1  27
2  68
both  101
ALL:  196
******************
TO: OK vs N2
df  193
df2  86
1  125
2  18
both  68
ALL:  211
******************
TO vs J2: OK vs N2
df  211
df2  196
1  104
2  89
both  107
ALL:  300
******************
OK vs OK: TO vs J2
df  193
df2  128
1  109
2  44
both  84
ALL:  237
******************


In [65]:
df_out = pd.concat([df_HS,df_N2,df_OK])
# df_out.to_csv("DoGs_ALL.csv", sep=",", index=None)

In [66]:
df = pd.concat([df_HS_J2,df_HS_TO,df_N2_J2,df_N2_TO,df_OK_J2,df_OK_TO])
# df.to_csv("DoGs_collapsed_by_enrichment_and_treatment.csv", sep=",", index=None)

In [67]:
df["Treatment_Enrichment"] = df["Treatment"] +"_"+ df["Enrichment"]
df.head()

,sample_name,DOG_length,gene_id_name,DOG_start_local,DOG_end_local,chr,strand,TYPE,Enrichment,Treatment,Treatment_Enrichment
315,N2HS_J2A,29300,WBGene00022951,16931619,16943323,X,+,DOG,J2,HS,HS_J2
763,N2HS_J2A,19500,WBGene00197215,10043561,10048167,V,-,DOG,J2,HS,HS_J2
1180,N2HS_J2A,16200,WBGene00002019,9088617,9090077,V,+,DOG,J2,HS,HS_J2
564,N2HS_J2A,15900,WBGene00219343,18723771,18724748,V,-,DOG,J2,HS,HS_J2
149,N2HS_J2A,15900,WBGene00002018,1805875,1807450,V,+,DOG,J2,HS,HS_J2


In [68]:
tlist = df["Treatment_Enrichment"].unique().tolist()
names = df["gene_id_name"].unique().tolist()
tlist

['HS_J2', 'HS_INP', 'N2_J2', 'N2_INP', 'OK_J2', 'OK_INP']

In [70]:


######UPSET For groups of samples
df_f = pd.DataFrame(columns=tlist)
df_f["gene_id_names"] = names
df_f = df_f.fillna(value=0)
df_f.index = df_f["gene_id_names"]

for name in names:
    df1 = df[df["gene_id_name"]==name]
    tl = df1["Treatment_Enrichment"].unique().tolist()
    for t in tl:
        df_f.loc[name,t] = 1 

final_names = ["gene_id_names"]+tlist
df_f = df_f[final_names]
df_f.to_csv("df_for_pyupset.csv",sep=",",index=None)

In [127]:

# slist = df["sample_name"].unique().tolist()
# # UPSET For groups of samples
# df_f = pd.DataFrame(columns=slist)
# df_f["gene_id_names"] = names
# df_f = df_f.fillna(value=0)
# df_f.index = df_f["gene_id_names"]

# for name in names:
#     df1 = df[df["gene_id_name"]==name]
#     tl = df1["sample_name"].unique().tolist()
#     for t in tl:
#         df_f.loc[name,t] = 1 

# final_names = ["gene_id_names"]+slist
# df_f = df_f[final_names]
# df_f.to_csv("df_for_pyupset_Allsamples.csv",sep=",",index=None)
    


In [71]:
slist

NameError: name 'slist' is not defined

In [129]:



# pd.DataFrame(tmp, columns=["First", "Second"]).set_index('First', drop=False)

In [72]:





hsup = ["WBGene00219916",
"WBGene00220094",
"WBGene00220004",
"WBGene00220243",
"WBGene00220000",
"WBGene00201688",
"WBGene00199162",
"WBGene00002017",
"WBGene00004567",
"WBGene00010790",
"WBGene00017659",
"WBGene00000766",
"WBGene00012977",
"WBGene00016002",
"WBGene00012663",
"WBGene00012734",
"WBGene00014115",
"WBGene00007027",
"WBGene00001749",
"WBGene00220163",
"WBGene00219826",
"WBGene00013024",
"WBGene00012718",
"WBGene00014954",
"WBGene00220246",
"WBGene00017245",
"WBGene00023180",
"WBGene00015800",
"WBGene00009277",
"WBGene00009052",
"WBGene00022167",
"WBGene00044131",
"WBGene00009527",
"WBGene00015731",
"WBGene00010770",
"WBGene00001889",
"WBGene00014264",
"WBGene00009654",
"WBGene00007637",
"WBGene00017499",
"WBGene00015228",
"WBGene00014309",
"WBGene00009656",
"WBGene00003229",
"WBGene00001923",
"WBGene00001901",
"WBGene00045127",
"WBGene00001234",
"WBGene00014843",
"WBGene00012961",
"WBGene00201669",
"WBGene00220052",
"WBGene00195813",
"WBGene00011231",
"WBGene00196396",
"WBGene00003308",
"WBGene00002015",
"WBGene00198814",
"WBGene00045366",
"WBGene00012555",
"WBGene00004765",
"WBGene00013732",
"WBGene00007859",
"WBGene00003834",
"WBGene00201398",
"WBGene00008968",
"WBGene00004905",
"WBGene00004314",
"WBGene00006945",
"WBGene00220207",
"WBGene00021238",
"WBGene00009301",
"WBGene00235348",
"WBGene00001918",
"WBGene00004246",
"WBGene00010713",
"WBGene00021317",
"WBGene00012362",
"WBGene00001925",
"WBGene00009050",
"WBGene00001895",
"WBGene00021236",
"WBGene00219803",
"WBGene00014376"]


hsdo = [   
"WBGene00235373",
"WBGene00220042",
"WBGene00269391",
"WBGene00000793",
"WBGene00200167",
"WBGene00012840",
"WBGene00043327",
"WBGene00012618",
"WBGene00016458",
"WBGene00017930",
"WBGene00194981",
"WBGene00008144",
"WBGene00164974",
"WBGene00019279",
"WBGene00009589",
"WBGene00012476",
"WBGene00004947",
"WBGene00198348",
"WBGene00010989",
"WBGene00197228",
"WBGene00022092",
"WBGene00077687",
"WBGene00199742",
"WBGene00019590",
"WBGene00043328"]


In [73]:
#####UPSET For groups of samples
df_f = pd.DataFrame(columns=tlist)
df_f["gene_id_names"] = names
df_f = df_f.fillna(value=0)
df_f.index = df_f["gene_id_names"]

for name in names:
    df1 = df[df["gene_id_name"]==name]
    tl = df1["Treatment_Enrichment"].unique().tolist()
    for t in tl:
        df_f.loc[name,t] = 1 

final_names = ["gene_id_names"]+tlist
df_f = df_f[final_names]


In [74]:
df_f.head()

,gene_id_names,HS_J2,HS_INP,N2_J2,N2_INP,OK_J2,OK_INP
gene_id_names,,,,,,,
WBGene00022951,WBGene00022951,1,0,1,1,1,1
WBGene00197215,WBGene00197215,1,1,0,0,0,0
WBGene00002019,WBGene00002019,1,0,0,0,0,0
WBGene00219343,WBGene00219343,1,0,0,0,0,0
WBGene00002018,WBGene00002018,1,0,0,0,0,0


In [75]:
df_up = pd.DataFrame()
df_up["gene_id_names"] = hsup

df_do = pd.DataFrame()
df_do["gene_id_names"] = hsdo

df_merge_up = pd.merge(df_f,df_up, on="gene_id_names")
df_merge_do = pd.merge(df_f,df_do, on="gene_id_names")

In [76]:
df_merge_up.to_csv("df_for_pyupset_hs_significant_up.csv",sep=",",index=None)
df_merge_do.to_csv("df_for_pyupset_hs_significant_do.csv",sep=",",index=None)

